#### HTML to JSON format

In [63]:
import re
import json

f1 = open('ohg_dict_12.html', 'r', encoding = 'utf-8')
f2 = open('ohg_dict_final.html', 'w', encoding = 'utf-8')

lines = f1.read().split('\n')
for line in lines:
    
    head_words = []
    sources = []
    linked_words = []
    linked_entries = []
    lex = []

    # heading words
    head = '<heading.*?</h'
    head_res = re.findall(head, line)
    
    for h in head_res:
        if 'variants' in h:
            
            var = re.findall('"(.*?)"', h)
            var = var[0].split(', ')
            
            if len(var) > 1:     
                for v in var:
                    head_words.append(v)
                    
        else:
            he = re.findall('strong>(.*?)</strong', h)
            head_words.append(he[0])
            
    # sources
    source_res = re.findall('<source>(.*?)</source>', line)
    if source_res:

        for s in source_res:
            s = s.replace('— ', '')
            s = s.replace('—', '')
            sources.append(s)
    
    # linked words
    links = re.findall('<linked(.*?)</linked>', line)
    if links:
        for l in links:
            
            if 'variants' in l: 
                varis = re.findall('="(.*?)"', l)
                
                for v in varis:
                    v = v.split(', ')
                    for word in v:
                        linked_words.append(word)
            
            else:
                l = l.replace('см. ', '')
                l = l.replace('>', '')
                linked_words.append(l)
    
    # linked entries (no duplicates)
    any_count = 0
    if linked_words:
        for w in linked_words:
            w_head1 = '<strong>' + w + '</strong>'
            w_head2 = 'heading variants=".*?' + w + '.*?><strong>'
            w_head2 = 'heading variants="(.*?)"><strong>'
            w_head3 = '<lex>' + w + '</lex>'
                        
            for line_1 in lines:
                w_count = 0            
                if w_head1 in line_1:
                    w_count += 1
                    any_count += 1

                var_res = re.findall(w_head2, line_1)
                if var_res:
                    for v in var_res:
                        v = v.split(', ')

                        for va in v:
                            if va == w:                        
                                w_count += 1
                                any_count += 1
                    
                if w_head3 in line_1:
                    w_count += 1
                    any_count += 1

                if w_count > 0:
                    line_1 = line_1.replace('<p>', '')
                    line_1 = line_1.replace('</p>', '')
                    linked_entries.append(line_1)
        linked_entries = list(set(linked_entries))
    
    # any <lex> words
    le = '<lex>(.*?)</lex>'
    le_var = '<lex variants="(.*?)">'
    
    le_res = re.findall(le, line)
    le_var_res = re.findall(le_var, line)
    
    for ler in le_res:
        lex.append(ler)
        
    for ler1 in le_var_res:
        ler1 = ler1.split(', ')
        for wor in ler1:
            lex.append(wor)
        
    lex = list(set(lex))
    
    entry = line.replace('<p>', '')
    entry = entry.replace('</p>', '')

    # prepare for json
    data = {}
    data['headers'] = head_words
    data['entry'] = entry
    data['sources'] = sources
    data['other words'] = lex
    data['linked words'] = linked_words
    data['linked entries'] = linked_entries
    
    json_string = json.dumps(data, ensure_ascii=False)
    
    all_strings.append(json_string)
    f2.write(json_string)
    f2.write('\n')
    
f1.close()
f2.close()

#### Working with 'ohg_dict_final.html'

In [65]:
import ast
with open("ohg_dict_final.html", 'r') as json_file:
    lines = json_file.read().split('\n')
    for line in lines:
        line = ast.literal_eval(line)
        for k, v in line.items():
            
            # do whatever with each element
            
            pass